In [ ]:
import matplotlib
from imshowpair import imshowpair
import matplotlib.pyplot as plt
import numpy as np
import os
from oiffile import imread
import random
import skimage
from skimage.transform import resize
import matplotlib
import glob
import torch
import sys
sys.path.insert(0, '..')
from other import dognet
from other import functions
from other.dognet.unet import *
import scipy.io as sio
import matplotlib.patches as patches
from skimage.measure import label,regionprops
from matplotlib_scalebar.scalebar import ScaleBar
from skimage.transform import rescale
from sklearn.metrics import roc_curve, auc
import os, os.path
%matplotlib inline
from skimage.draw import circle
import imageio

In [ ]:
# get training masks
path1 = glob.glob("../../datasets/flocculusA/trainingMasks/*.npy")
trainingMasks = functions.get_trainingMasks(path1)
# get and scale training images
path2 = glob.glob("../../datasets/flocculusA/trainingImages/*.npy")
trainingImages,collman = functions.get_trainingImages(path2,20,25,25)

# randomly divide into training and testing images
rndsplit = np.random.permutation(len(trainingImages))
numTest = int(len(trainingImages)/4)
numTrain = int(len(trainingImages)-numTest)
imagestraining = rndsplit[:numTrain]
imagestesting = rndsplit[numTrain:]

newTrainingImages=[]
newTrainingMasks=[]
newTestingImages=[]
newTestingMasks=[]

for i in imagestraining:
    newTrainingImages.append(trainingImages[i])
    newTrainingMasks.append(trainingMasks[i])
for j in imagestesting:  
    newTestingImages.append(trainingImages[j])
    newTestingMasks.append(trainingMasks[j])
    
lossFn = "sd"
alpha = 0.5
gamma = 1
lrs = 1e-4
batchSize = 2
nEpoch = 1200
dschedule = [50,0.98]
# save networks, losses, and test images over epochs
networks = []
trainingLoss = []
testingLoss = []
visualTests = []
print("training images:")
print(imagestraining)
print("testing images:")
print(imagestesting)

In [ ]:
# train 10 networks
for i in range(0,10):
    # train network
    device = torch.device('cuda')
    net = UNet(3,2).to(device)
    net,train_errors,test_errors,visual_test=dognet.train_routine(net,dognet.create_generator(newTrainingImages,newTrainingMasks,device = device,batch_size = batchSize),dognet.create_generator(newTestingImages,newTestingMasks,device = device,batch_size = batchSize,transform=False),n_train_samples = numTrain, n_test_samples=numTest,n_epoch=nEpoch,batch_size=batchSize,loss=lossFn,lr=lrs,alpha = alpha,decay_schedule = dschedule,gamma = gamma,device = device)
    
    networks.append(net)
    del net
    trainingLoss.append(train_errors)
    testingLoss.append(test_errors)
    new_visual_test=[]
    # save test image every 5 epochs
    for x in range(0, int(1200/5)):
        new_visual_test.append(visual_test[5*x])
    visualTests.append(new_visual_test)

In [ ]:
# view test image, average predicted mask, annotated mask
for selidx in range(0,numTest,1):
    if np.sum(newTestingMasks[selidx])>0:
        plt.figure(figsize=(9,3))
        inimg = np.transpose(resize(np.transpose(newTestingImages[selidx],(1,2,0)),(230,230)),(2,0,1))
        ims = []
        for netnum in range(0,10):
            outname = functions.inference(networks[netnum].to(torch.device("cuda")),inimg,get_inter=False,device=torch.device("cuda"))
            ims.append(outname)
        plt.subplot(1,3,1)
        plt.imshow(np.clip(np.transpose(newTestingImages[selidx],(1,2,0)),0,1))
        plt.clim(0,1)
        plt.axis('off')
        plt.subplot(1,3,2)
        plt.imshow(np.median(np.asarray(ims)[:,0,:,:], axis=0),cmap='gray')
        plt.clim(0,.8)
        plt.axis('off')
        plt.subplot(1,3,3)
        plt.imshow(newTestingMasks[selidx],cmap='gray')
        plt.clim(0,1)
        plt.axis('off')
        plt.show()

In [ ]:
#save networks
for i in range(0,10):
    nsave = networks[i]
    name = "../networks/multipleNetworks-flocculusA/net%s" % (i)
    torch.save(nsave, name)

In [ ]:
# view test and training error graphs
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(np.arange(nEpoch+1),np.asarray(trainingLoss).T,'grey')
plt.plot(np.arange(nEpoch+1),np.mean(np.asarray(trainingLoss).T,1),'r-')
plt.xlabel('Epoch')
plt.ylabel('Softdice Loss')
plt.title("Training Loss")
plt.ylim(0,0.005)
plt.xlim(0,nEpoch)
plt.subplot(1,2,2)
plt.plot(np.arange(nEpoch+1),np.asarray(testingLoss).T,'grey')
plt.plot(np.arange(nEpoch+1),np.mean(np.asarray(testingLoss).T,1),'b-')
plt.xlabel('Epoch')
plt.ylabel('Softdice Loss')
plt.title("Testing Loss")
plt.ylim(0,0.005)
plt.xlim(0,nEpoch)
plt.show()

In [ ]:
# view average test and training loss
plt.figure(figsize=(5.5,4))
plt.plot(np.arange(nEpoch+1),np.mean(np.asarray(trainingLoss).T,1),'r-',label='training')
plt.plot(np.arange(nEpoch+1),np.mean(np.asarray(testingLoss).T,1),'b-',label='testing')
plt.xlabel('Epoch')
plt.ylabel('Softdice Loss')
plt.title("Training and Testing Losses")
plt.ylim(0,0.005)
plt.xlim(0,nEpoch)
plt.legend()
plt.show()

In [ ]:
# save gif of predicted mask of average test image over epochs
images =[]
for j in range(0,240):
    ims = []
    for i in range(0,10):
        ims.append(visualTests[i][j])
    images.append(( (sum(ims)/len(ims)) .detach().numpy()))
    
imageio.mimsave('gifName.gif', images, fps = 20)